In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf 

In [ ]:
train_dir = '/kaggle/input/cat-and-dog/training_set/training_set'
validation_dir = '/kaggle/input/cat-and-dog/test_set/test_set'
image_size = 128
batch_size = 32
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(directory=train_dir, target_size=(image_size, image_size), batch_size=batch_size)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(directory=validation_dir, target_size=(image_size, image_size), batch_size=batch_size)

In [ ]:
IMG_SHAPE = (image_size, image_size, 3)
base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, include_top=False)
base_model.trainable = False
model = tf.keras.Sequential([
 base_model,
 tf.keras.layers.GlobalAveragePooling2D(),
 tf.keras.layers.Dense(2, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
 loss='categorical_crossentropy',
 metrics=['accuracy'])
epochs = 25
steps_per_epoch = np.asarray(train_generator.n / batch_size)
validation_steps = np.asarray(validation_generator.n / batch_size)
history = model.fit_generator(generator=train_generator,
 steps_per_epoch=steps_per_epoch,
 epochs=epochs,
 validation_data=validation_generator,
 validation_steps=validation_steps)

In [ ]:
saved_model_dir = '../assets'
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open('model17.tflite', 'wb') as f:
    f.write(tflite_model)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels17.txt', 'w') as f:
    f.write(labels)

In [ ]:
import os
os.chdir(r'/kaggle/working')

os.stat('model17.tflite').st_size

In [ ]:
# from IPython.display import FileLink
# FileLink(r'model17.tflite')